In this notebook, we will use information-matching method to find the optimal locations of hydrophones to simultaneously infer the environmental parameters and localize the sound sources.
We will use the transmission loss data from both the top and bottom sound sources.

For the source locations, we require precision within $\pm$ 2.5 m for the depth and $\pm$ 100 m for the range.
However, we don't really care about the environmental parameters.
Thus, we will set the target error bars for the environmental parameter to infinity.

In [ ]:
from pathlib import Path
import itertools
import pickle

import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import matplotlib as mpl
from svgpathtools import svg2paths
from svgpath2mpl import parse_path

# from fimpack import orca_manager

from information_matching.convex_optimization import ConvexOpt
from information_matching.utils import tol, eps, set_directory
from information_matching.summary import Summary

%matplotlib inline
plt.style.use("default")

In [ ]:
# Setting up directories
WORK_DIR = Path().absolute()
DATA_DIR = WORK_DIR / "data"
SVP_DIR = DATA_DIR / "svp"
TL_DIR = DATA_DIR / "transmission_loss"
FIM_DIR_SOURCE = DATA_DIR / "FIMs" / "source"
FIM_DIR_ENV = DATA_DIR / "FIMs" / "environment"
ERROR_DIR = DATA_DIR / "target_error" / "source_environment"
RESULT_DIR = WORK_DIR / "results" / ERROR_DIR.name

In [ ]:
# List of TOML files
opt_toml_file = DATA_DIR / "FIM_opt.toml"
# Sound profile files
sediment_type_list = ["mud", "clay", "silt", "sand", "gravel"]
svp_toml_path_dict = {
    sed: SVP_DIR / f"svp_{sed}_35m_unit_test.toml" for sed in sediment_type_list
}
# List of sound source frequencies
freq_list = [50, 100, 200, 400]  # In Hz

# Setup

In [ ]:
# This cell has variables that control the environment setup
ised, ifreq = 0, 0  # Index sediment type and frequency, refer to the lists abovev
sediment_type = sediment_type_list[ised]
freq = freq_list[ifreq]  # in Hz

# Prepare a summary file
CASE_DIR = set_directory(RESULT_DIR / sediment_type / f"f{freq}Hz")
summary = Summary(CASE_DIR / "summary.json")

In [ ]:
# Define the configurations
source_depth = np.array([8, 16])
source_range = np.linspace(1, 5, 21)
receiver_depth = receiver_depth = np.arange(5, 76, 5)
configs = list(itertools.product(source_range, receiver_depth))

# Don't use receiver depth = 75
h_ocean = 75
idx = np.where(np.array(configs)[:, 1] < h_ocean)[0]
configs = [configs[ii] for ii in idx]
nconfigs = len(configs)
config_ids = [f"range_{int(conf[0]*1000)}m_depth_{conf[1]}m" for conf in configs]
print("Number of configurations:", len(configs))

## Target FIM

In [ ]:
# The following is for plotting
param_names = [
    r"$d_{s1}$",
    r"$r_{s1}$",
    r"$d_{s2}$",
    r"$r_{s2}$",
    r"$h_{ocean}$",
    r"$c^1_{ocean}$",
    r"$c^2_{ocean}$",
    r"$h_{sed}$",
    r"$c^1_{sed}$",
    r"$c^2_{sed}$",
    r"$\rho^1_{sed}$",
    r"$\rho^2_{sed}$",
    r"$\alpha^1_{sed}$",
    r"$\alpha^2_{sed}$",
    r"$c_{base}$",
    r"$\rho_{base}$",
    r"$\alpha_{base}$",
]
nparams = len(param_names)

In [ ]:
# Get target error
target_error_file = ERROR_DIR / f"target_error_{sediment_type}_f{freq}Hz.npz"
if target_error_file.exists():
    bestfit_error = np.load(target_error_file)
    best_fit = bestfit_error["bestfit"]
    target_error = bestfit_error["error"]
else:
    # Initialize ORCA
    svp_toml_path = svp_toml_path_dict[sediment_type]
    orca = orca_manager.initialize_orca(freq, str(svp_toml_path), str(opt_toml_file))
    x_dict, _ = orca_manager.get_x_dict(orca)
    # Best fit
    best_fit_source = np.array([source_depth[0], 0.0, source_range[1], 0.0])
    best_fit_env = np.array(
        [
            x_dict[name]
            for name in [
                "h_ocean",
                "cp1_ocean",
                "cp2_ocean",
                "h_layer1",
                "cp1_layer1",
                "cp2_layer1",
                "rho1_layer1",
                "rho2_layer1",
                "ap1_layer1",
                "ap2_layer1",
                "cp1_hsp",
                "rho1_hsp",
                "ap1_hsp",
            ]
        ]
    )
    best_fit = np.append(best_fit_source, best_fit_env)
    # Target error bars
    target_error_source = np.array([2.5, 1e-1, 2.5, 1e-1])
    target_error_env = np.ones_like(best_fit_env) * np.inf
    target_error = np.append(target_error_source, target_error_env)
    # Export
    np.savez(target_error_file, bestfit=best_fit, error=target_error)

print("Target error:", target_error)

In [ ]:
# Set the target FIM
fim_target = np.diag(np.append(1 / target_error[:4] ** 2, np.zeros(nparams - 4)))
lambdas_target, v_target = np.linalg.eigh(fim_target)
# This indices are to help comparing the eigenvalues later
idx_sort = np.argsort(target_error)

## Load configuration FIMs

In [ ]:
# Collect the FIMs of the configurations
fim_configs_tensor = np.empty((nconfigs, nparams, nparams))
for ii in range(nconfigs):
    # Jacobian wrt sources
    jac_source = np.load(
        FIM_DIR_SOURCE / sediment_type / f"f{freq}Hz" / f"config_{ii}.npz"
    )["jacobian"]
    # Jacobian wrt environment parameters
    jac_env = np.load(
        FIM_DIR_ENV / sediment_type / f"f{freq}Hz" / f"config_{ii}.npz"
    )["jacobian"]
    # Combine the Jacobian
    jac = np.column_stack((jac_source, jac_env))
    fim_configs_tensor[ii] = jac.T @ jac

# Convex optimization

In [ ]:
# Construct the input FIMs
# FIM target
norm = np.linalg.norm(fim_target)
fim_target = {"fim": fim_target, "scale": 1 / norm}
# FIM configs
fim_configs = {}
for ii, identifier in enumerate(config_ids):
    norm = np.linalg.norm(fim_configs_tensor[ii])
    fim_configs.update(
        {identifier: {"fim": fim_configs_tensor[ii], "fim_scale": 1 / norm}}
    )

In [ ]:
# Convex optimization
# Settings
cvx_tol = tol
solver = dict(verbose=True, solver=cp.SDPA, epsilonStar=cvx_tol, lambdaStar=1e-1)
cvxopt = ConvexOpt(fim_target, fim_configs)

# Solve
cvxopt_file = CASE_DIR / "cvx_result.pkl"
print("Tolerance:", cvx_tol)
if cvxopt_file.exists():
    with open(cvxopt_file, "rb") as f:
        cvxopt.result = pickle.load(f)
    print("Violation:", cvxopt.result["violation"])
    print("Eigenvalues of the difference matrix:")
    print(
        np.linalg.eigvalsh(
            cvxopt._difference_matrix(cvxopt.result["wm"].reshape((-1, 1))).value
        )
    )
else:
    while solver["lambdaStar"] < 1e3:
        try:
            cvxopt.solve(**solver)
            with open(cvxopt_file, "wb") as f:
                pickle.dump(cvxopt.result, f)
            # The positive defininte condition is not strictly satisfied; only satisfied within
            # the tolerance.
            print("Violation:", cvxopt.constraints[1].violation())
            print("Eigenvalues of the difference matrix:")
            print(
                np.linalg.eigvalsh(
                    cvxopt._difference_matrix(
                        cvxopt.result["wm"].reshape((-1, 1))
                    ).value
                )
            )
            break
        except Exception:
            solver["lambdaStar"] *= 10

# Update the summary file
summary.update(cvxopt.result, "convex optimization")

## Extracting the weights

In [ ]:
wm = cvxopt.result["wm"]
unscaled_wm = cvxopt._get_unscaled_weights(wm)
dw = cvxopt.result["dual_wm"]
wtol = cvx_tol ** 0.5

# Plot the weights
plt.figure()
plt.plot(cvxopt.result["wm"], label="weights")
plt.plot(cvxopt.result["dual_wm"], label="dual weights")
# plt.axhline(wtol, color="k")
plt.yscale("log")
plt.legend()
# plt.savefig(CASE_DIR / "opt_weights.png")
plt.show()

In [ ]:
# Issue: In some cases, the dual values of the dominant weights are not significantly
# smaller than the others.As such, we might have too few configurations if we use the
# criteria above.
# We can always add more configurations to constrain the parameters more. Thus, I want
# to add this modification in the criteria to select optimal configurations. I will
# add more and more configurations, from the ones with largest weights, until the
# optimal errors are smaller than the target.
# I think I can achieve this since the positive definite violation is small.
def compute_fim_configs(idx):
    weights_all = np.zeros(nconfigs)
    for ii in idx:
        weights_all[ii] = unscaled_wm[ii]
    fim_configs = np.sum(
        fim_configs_tensor * weights_all.reshape((nconfigs, 1, 1)), axis=0
    )
    return fim_configs


def compute_optimal_error(fim):
    cov = np.linalg.pinv(fim)
    err = np.sqrt(np.diag(cov))
    return err

In [ ]:
# Get the optimal weights by comparing the dual value of the weights
idx_nonzero_weights = np.where(dw < wtol)[0]
opt_weights = cvxopt._get_config_weights_from_idx(idx_nonzero_weights)
print("Number of optimal configurations", len(opt_weights))
with open(CASE_DIR / "configs_weights.pkl", "wb") as f:
    pickle.dump(opt_weights, f)
summary.update(opt_weights, "reduced configurations weights")
opt_weights

In [ ]:
idx_sort_wm = np.argsort(unscaled_wm)[::-1]

for n in range(1, nconfigs + 1):
    idx_list = idx_sort_wm[:n]
    for ii in idx_list:
        if ii not in idx_nonzero_weights:
            idx_nonzero_weights = np.append(idx_nonzero_weights, ii)
    fim_configs = compute_fim_configs(idx_nonzero_weights)
    lambdas_configs, v = np.linalg.eigh(fim_configs)
    # First, check if the eigenvalues of the configuration FIM are all larger than the
    # eigenvalues of the target FIM
    if not np.all(lambdas_configs - lambdas_target > 0):
        continue

    diff_error = compute_optimal_error(fim_configs) - target_error
    print(diff_error)
    # Check if the optimal error are all smaller than the target (our goal)
    if np.all(diff_error < 0):
        print("Strictly satisfied")
        break
    else:
        nviolated = nparams - np.sum(diff_error < 0)
        if nviolated <= 3:
            # Due to some violation in the positive semidefinite condition, the goal might
            # not be achievable, but the error difference should be very small. We will
            # still accept it if the error difference is below some small threshold (1e-2)
            loc_viol = np.where(diff_error > 0)[0]
            if np.all(np.abs(diff_error[loc_viol] / target_error[loc_viol]) < 10.0):
                print("Non-strictly satisfied")
                break

print("Number of optimal configurations", len(idx_nonzero_weights))
opt_weights = cvxopt._get_config_weights_from_idx(idx_nonzero_weights)
with open(CASE_DIR / "configs_weights.pkl", "wb") as f:
    pickle.dump(opt_weights, f)
summary.update(opt_weights, "reduced configurations weights")
opt_weights

# Analyze the result

In [ ]:
# Get the final configuration FIM. Note that we only use the nonzero optimal weights.
# idx_nonzero_weights = cvxopt.get_idx_nonzero_wm(weight_tol)
config_ids_nonzero_weights = [config_ids[ii] for ii in idx_nonzero_weights]
nonzero_weights = np.array([opt_weights[name] for name in config_ids_nonzero_weights])
fim_configs = compute_fim_configs(idx_nonzero_weights)
lambdas_configs, v = np.linalg.eigh(fim_configs)
np.save(CASE_DIR / "fim_configs.npy", fim_configs)

## Eigenvalues of the FIMs

In [ ]:
# Plot the eigenvalues
plt.figure()
for lt, lc in zip(lambdas_target, lambdas_configs):
    plt.plot([-0.5, 0.5], [lt, lt], "-", c="tab:blue")
    plt.plot([0.5, 1.5], [lt, lc], "--", c="k")
    plt.plot([1.5, 2.5], [lc, lc], "-", c="k")
plt.yscale("log")
plt.xticks([0, 2], ["Target QoI", "Configurations"])
plt.ylabel("Eigenvalues")
# plt.savefig(CASE_DIR / "eigenvalues.png")
plt.show()

## Parameter uncertainty

In [ ]:
# Plot the error result
opt_error = compute_optimal_error(fim_configs)

plt.figure(dpi=300)
plt.plot(target_error, "-o", label="target")
plt.plot(opt_error, "-o", label="optimal")
plt.xticks(range(nparams), param_names, rotation=90)
plt.ylim(1e-4, 1e3)
plt.ylabel("Error")
plt.yscale("log")
plt.legend()
plt.savefig(CASE_DIR / "error_qoi.png")
plt.show()

## Visualize the result on the real environment

In [ ]:
# For plotting
speaker_path, attributes = svg2paths(DATA_DIR / "Speaker_Icon.svg")
speaker_marker = parse_path(attributes[0]["d"])
# Touchups
speaker_marker.vertices -= speaker_marker.vertices.mean(axis=0)
speaker_marker = speaker_marker.transformed(mpl.transforms.Affine2D().rotate_deg(180))
speaker_marker = speaker_marker.transformed(mpl.transforms.Affine2D().scale(-1,1))

In [ ]:
# Transmission loss data for plotting
TL_file = TL_DIR / f"TL_{sediment_type}_f{freq}Hz.npz"
TL_data = np.load(TL_file)
source_range_fine = TL_data["source_range"]
receiver_depth_fine = TL_data["receiver_depth"]
TL = TL_data["TL"]    
SR, RD = np.meshgrid(source_range_fine, receiver_depth_fine)

In [ ]:
configs = np.array(configs)
# TL from the top source
plt.figure(dpi=300, figsize=(6.4, 9.6))

# Plot transmission loss profile
plt.contourf(SR, -RD, TL[0, :, :, 0], levels=np.linspace(-132, -16, 117))
plt.colorbar(label="Transmission loss", orientation="horizontal", pad=0.075)

# Plot the halfspace/basement
plt.axhline(-h_ocean, c="k")
plt.fill_between(
    [-0.1, source_range[-1]],
    [-h_ocean, -h_ocean],
    [-h_ocean - 10, -h_ocean - 10],
    color="brown",
)
plt.text(
    2.0,
    -np.max(receiver_depth) - 5,
    "Sediment",
    bbox={"facecolor": "white", "ec": "white"},
)

# Put the sources
plt.errorbar(
    [0],
    -source_depth[0],
    yerr=target_error[[0]],
    xerr=target_error[[1]],
    marker=speaker_marker,
    markersize=20,
    color="blue",
    capsize=5,
)

# Plot all receivers
# plt.plot(configs[:, 0], -configs[:, 1], "ko")
# Plot the results
plt.plot(
    configs[idx_nonzero_weights, 0],
    -configs[idx_nonzero_weights, 1],
    "ko",
    label="Optimal receiver",
)

plt.xlim(-0.2, source_range[-1])
plt.ylim(-receiver_depth[-1] - 10, 0)

plt.xlabel("Range (km)")
plt.ylabel("Depth (m)")
# plt.legend()
plt.savefig(CASE_DIR / "optimal_configs_top.png")
plt.show()

In [ ]:
# TL from the bottom source
plt.figure(dpi=300, figsize=(6.4, 9.6))

# Plot transmission loss profile
plt.contourf(SR, -RD, TL[1, :, :, 0], levels=np.linspace(-132, -16, 117))
plt.colorbar(label="Transmission loss", orientation="horizontal", pad=0.075)

# Plot the halfspace/basement
plt.axhline(-h_ocean, c="k")
plt.fill_between(
    [-0.1, source_range[-1]],
    [-h_ocean, -h_ocean],
    [-h_ocean - 10, -h_ocean - 10],
    color="brown",
)
plt.text(
    2.0,
    -np.max(receiver_depth) - 5,
    "Sediment",
    bbox={"facecolor": "white", "ec": "white"},
)

# Put the sources
plt.errorbar(
    [0],
    -source_depth[1],
    yerr=target_error[[2]],
    xerr=target_error[[3]],
    marker=speaker_marker,
    markersize=20,
    color="blue",
    capsize=5,
)

# Plot all receivers
# plt.plot(configs[:, 0], -configs[:, 1], "ko")
# Plot the results
plt.plot(
    configs[idx_nonzero_weights, 0],
    -configs[idx_nonzero_weights, 1],
    "ko",
    label="Optimal receiver",
)

plt.xlim(-0.2, source_range[-1])
plt.ylim(-receiver_depth[-1] - 10, 0)

plt.xlabel("Range (km)")
plt.ylabel("Depth (m)")
# plt.legend()
plt.savefig(CASE_DIR / "optimal_configs_bottom.png")
plt.show()